# B6644673 นายภาณุเดช ศรีวุฒิทรัพย์

### คำถาม Challenge:

1. **Planning Optimization:** หาวิธีจัดงานปาร์ตี้ที่ประหยัดที่สุดโดยใช้เวลาไม่เกิน 8 ชั่วโมง

    **ANS** ซื้ออาหารครบ + เครื่องดื่ม + ตกแต่ง + เพลง + เชิญแขก
            💰 $320 | ⏱ 7 ชม. | เหลือเงิน $180 เหลือเวลา 1 ชม.
2. **Resource Management:** ถ้าเงินมีแค่ $300 จะต้องปรับแผนอย่างไร?

    **ANS** 👉 ซื้ออาหาร 2 อย่าง + ทำเค้กเอง + เครื่องดื่ม + ตกแต่ง + เพลง + เชิญแขก
            💰 $280 | ⏱ 7 ชม.
3. **Parallel Planning:** งานไหนบ้างที่สามารถทำพร้อมกันได้?

    **ANS** ซื้อของ + จัดเพลง + เชิญแขก ทำพร้อมกันได้
4. **Algorithm Selection:** ปัญหานี้เหมาะกับอัลกอริทึมไหน? GraphPlan, POP, หรือ Hierarchical Planning? ทำไม?

    **ANS** ใช้ GraphPlan เหมาะสุด เพราะมี dependency + resource constraints + ทำงานขนานได้

In [13]:
from planning import PlanningProblem, Action, expr

def party_planning_problem():
    """
    🏆 Challenge: Party Planning Problem
    """
    # Initial State
    init = [
        expr('Money500'),          
        expr('Time8'),             
        expr('~Decorated'),
        expr('~HasDecorations'),
        expr('~HasFoodPizza'),
        expr('~HasFoodSandwich'),
        expr('~HasFoodCake'),
        expr('~HasDrinks'),
        expr('~MusicReady'),
        expr('~GuestsInvited')
    ]
    
    # Goals
    goals = [
        expr('Decorated'),
        expr('HasFoodPizza'),
        expr('HasFoodSandwich'),
        expr('HasFoodCake'),
        expr('HasDrinks'),
        expr('MusicReady'),
        expr('GuestsInvited')
    ]
    
    # Actions (enumerated literals)
    actions = [
        Action('BuyDecorations',
               precond=expr('Money500 & Time8'),
               effect=expr('HasDecorations & ~Money500 & Money400 & ~Time8 & Time6')),
        
        Action('Decorate',
               precond=expr('HasDecorations & Time6'),
               effect=expr('Decorated & ~Time6 & Time3')),
        
        Action('BuyFoodPizza',
               precond=expr('Money400'),
               effect=expr('HasFoodPizza & ~Money400 & Money350')),
        
        Action('BuyFoodSandwich',
               precond=expr('Money350'),
               effect=expr('HasFoodSandwich & ~Money350 & Money320')),
        
        Action('BuyFoodCake',
               precond=expr('Money320'),
               effect=expr('HasFoodCake & ~Money320 & Money240')),
        
        Action('BuyDrinks',
               precond=expr('Money240'),
               effect=expr('HasDrinks & ~Money240 & Money180')),
        
        Action('SetupMusic',
               precond=expr('Time3'),
               effect=expr('MusicReady & ~Time3 & Time2')),
        
        Action('InviteGuests',
               precond=expr('Time2'),
               effect=expr('GuestsInvited & ~Time2 & Time1'))
    ]
    
    return PlanningProblem(init, goals, actions)


# ---------- Helper Functions ----------
def parse_money(state):
    for s in state:
        if str(s).startswith("Money") and not str(s).startswith("Not"):
            return int(str(s).replace("Money",""))
    return 0

def parse_time(state):
    for s in state:
        if str(s).startswith("Time") and not str(s).startswith("Not"):
            return int(str(s).replace("Time",""))
    return 0

def can_apply(prob, action):
    """เช็ค preconditions แบบง่าย (ต้องการ literal ทั้งหมดอยู่ใน state)"""
    act_obj = None
    for a in prob.actions:
        if str(a.name) == str(action):
            act_obj = a
            break
    if act_obj is None:
        return False
    
    # precond อาจเป็น expr หรือ list → แปลงเป็น list
    preconds = act_obj.precond if isinstance(act_obj.precond, list) else [act_obj.precond]
    
    # เช็คว่าทุก precondition อยู่ใน state
    return all(p in prob.initial for p in preconds)


# ---------- Run the Party Plan ----------
print("🏆 Party Planning Challenge")
party_prob = party_planning_problem()
print("Initial:", party_prob.initial)
print("Goals:", party_prob.goals)
print("Actions:", len(party_prob.actions))

solution = [
    expr('BuyDecorations'),
    expr('Decorate'),
    expr('BuyFoodPizza'),
    expr('BuyFoodSandwich'),
    expr('BuyFoodCake'),
    expr('BuyDrinks'),
    expr('SetupMusic'),
    expr('InviteGuests')
]

print("\n=== Running Plan ===")
for i, action in enumerate(solution, 1):
    if can_apply(party_prob, action):
        party_prob.act(action)
        status = "✅ Done"
    else:
        status = "❌ Skipped (precond not met)"
    money = parse_money(party_prob.initial)
    time  = parse_time(party_prob.initial)
    print(f"Step {i}: {action} -> {status}")
    print(f"   Current Money: ${money}, Time: {time} hrs")

print("\n🏁 Final Result")
print(f"Total Money Spent: ${500 - parse_money(party_prob.initial)}")
print(f"Money Remaining: ${parse_money(party_prob.initial)}")
print(f"Total Time Used: {8 - parse_time(party_prob.initial)} hrs")
print(f"Time Remaining: {parse_time(party_prob.initial)} hrs")
print("Success:", party_prob.goal_test())


🏆 Party Planning Challenge
Initial: [Money500, Time8, NotDecorated, NotHasDecorations, NotHasFoodPizza, NotHasFoodSandwich, NotHasFoodCake, NotHasDrinks, NotMusicReady, NotGuestsInvited]
Goals: [Decorated, HasFoodPizza, HasFoodSandwich, HasFoodCake, HasDrinks, MusicReady, GuestsInvited]
Actions: 8

=== Running Plan ===
Step 1: BuyDecorations -> ✅ Done
   Current Money: $400, Time: 6 hrs
Step 2: Decorate -> ✅ Done
   Current Money: $400, Time: 3 hrs
Step 3: BuyFoodPizza -> ✅ Done
   Current Money: $350, Time: 3 hrs
Step 4: BuyFoodSandwich -> ✅ Done
   Current Money: $320, Time: 3 hrs
Step 5: BuyFoodCake -> ✅ Done
   Current Money: $240, Time: 3 hrs
Step 6: BuyDrinks -> ✅ Done
   Current Money: $180, Time: 3 hrs
Step 7: SetupMusic -> ✅ Done
   Current Money: $180, Time: 2 hrs
Step 8: InviteGuests -> ✅ Done
   Current Money: $180, Time: 1 hrs

🏁 Final Result
Total Money Spent: $320
Money Remaining: $180
Total Time Used: 7 hrs
Time Remaining: 1 hrs
Success: True
